## Facility location problem 

In [1]:
import numpy as np
import pandas as pd
import os
import networkx as nx
import random
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

from pyomo.environ import *

In [ ]:
random.seed(42)
plt.close('all')


# Dimensions of domain
X_min = -10
X_max = 10
Y_min = -10
Y_max = 10

# Costs (divided by 1,000)
# Min, max cost to build a facility
C_min = 10000 
C_max = 20000 
# Cost per unit distance when serving a customer
C_unit_distance = [200,500]


number_customers  = 100
number_facilities = 10 

############
### Sets ###
############

N = {k:{'x':np.round(X_min+(X_max-X_min)*random.random(),1),
        'y':np.round(Y_min+(Y_max-Y_min)*random.random(),1)}
         for k in range(1,number_customers+1)}

F = {k:{'x':np.round(X_min+(X_max-X_min)*random.random(),1),
        'y':np.round(Y_min+(Y_max-Y_min)*random.random(),1),
        'C':int(np.round(C_min+(C_max-C_min)*random.random(),0))}
         for k in range(1,number_facilities+1)}

# Plot demand nodes and facilities
axis_font  = {'fontname':'Arial', 'size':'15'}
fig, ax = plt.subplots()
    
for i,v_i in N.items():
    plt.plot(v_i['x'],v_i['y'],marker='o',markersize=5,color='k',zorder=10)

for f,v_f in F.items():
    plt.plot(v_f['x'],v_f['y'],marker='s',markersize=10,color='b',zorder=10)
    plt.text(v_f['x'],v_f['y'],str(f),color='r',fontsize=20,zorder=20)
        
ax.grid(True)
ax.set_xlabel('X',**axis_font)
ax.set_ylabel('Y',**axis_font)
plt.show()
fig.savefig('fac_loc_input.png', format='png', dpi=1000, bbox_inches='tight',
         transparent=True,pad_inches=0.02)

In [3]:
###################
#### Parameters ###
###################

D_if = {(i,f):np.sqrt((v_i['x']-v_f['x'])**2+(v_i['y']-v_f['y'])**2) 
        for i,v_i in N.items() for f,v_f in F.items()}


for c_unit in C_unit_distance:

    model = ConcreteModel()
        
    # Define sets
    model.Demand_nodes = Set(initialize=N.keys())
    model.Facilities   = Set(initialize=F.keys())
    
    # Define parameters
    model.D_if = Param(model.Demand_nodes,model.Facilities, initialize={(i,f):D_if[(i,f)] 
          for i in N.keys() for f in F.keys()}, within=NonNegativeReals)
    model.C_f = Param(model.Facilities, initialize={f:F[f]['C'] 
          for f in F.keys()}, within=NonNegativeReals)
    
    # Define decision variables
    model.y     = Var(model.Facilities, within=Binary)
    model.x     = Var(model.Demand_nodes,model.Facilities, within=Binary)
    
    # Define objective function
    model.obj = Objective(expr=sum(c_unit*model.D_if[i,f]*model.x[i,f] for i in model.Demand_nodes
                                   for f in model.Facilities)+sum(model.C_f[f]*model.y[f] for f in model.Facilities), sense=minimize)
    
    # Define constraints
    model.demand_node_assigned_once = ConstraintList()
    for i in model.Demand_nodes:
        model.demand_node_assigned_once.add(expr=sum(model.x[i,f] 
                                             for f in model.Facilities)==1)
        
    model.activation_facility = ConstraintList()
    for f in model.Facilities:
        model.activation_facility.add(expr=sum(model.x[i,f] for i in model.Demand_nodes)
                                      <= len(N)*model.y[f])
    
    model.write('facility_location.lp', io_options={'symbolic_solver_labels': True})
    
    # Solve the problem
    solver = SolverFactory('gurobi')
    solver.solve(model)
    
    # Print the results
    print('')
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Overall cost:', np.round(model.obj(),1))
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('')
    objective_p.append(np.round(model.obj(),1))
    
    print('%%%%%%%%%%%%%%%%%%%')
    print('Used facilities:')
    print('%%%%%%%%%%%%%%%%%%%')
    used_facilities = []
    for f in model.Facilities:
        if model.y[f].value >= 0.99:
            used_facilities.append(f)
    print(used_facilities)
    
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Assignment of demand nodes to facilities:')
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    N_f = {k:[] for k in F.keys()}
    for i in model.Demand_nodes:
        for f in model.Facilities:
            if model.x[(i,f)].value >= 0.99:
                print(f'Node {i} - Facility {f}')
                N_f[f].append(i)
    
    axis_font  = {'fontname':'Arial', 'size':'15'}            

    fig, ax = plt.subplots()
    
    for i,v_i in N.items():
        plt.plot(v_i['x'],v_i['y'],marker='o',markersize=5,color='k',zorder=10)
    
    for f,v_f in F.items():
        if f in used_facilities:
            plt.plot(v_f['x'],v_f['y'],marker='s',markersize=10,color='g',zorder=10)
        else:
            plt.plot(v_f['x'],v_f['y'],marker='s',markersize=10,color='r',zorder=10)
            
    
    for f in used_facilities:
        for i in N_f[f]:
            plt.plot([N[i]['x'],F[f]['x']],
                     [N[i]['y'],F[f]['y']],linewidth=1.5,color='b')
    
    ax.grid(True)
    ax.set_xlabel('X',**axis_font)
    ax.set_ylabel('Y',**axis_font)
    plt.show()
    fig.savefig('sol_facloc_%i.png'%(c_unit), format='png', dpi=1000, bbox_inches='tight',
             transparent=True,pad_inches=0.02)


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Overall cost: 147296.7
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%



NameError: name 'objective_p' is not defined